# **Training DNN and Bi-LSTM Models for fake news type classification**

Two datasets:

1. All Fakes = 1:"Disinformation", 2:"Hoax", 3:"Propaganda", 4:"Trusted"

2. TOVS =  1:"Satire", 2:"Hoax", 3:"Propaganda", 4:"Trusted"

In [30]:
import numpy as np
import os
import pickle
import tensorflow as tf
#import tensorflow_addons as tfa
import pandas as pd

In [32]:
intermed_path = 'intermediate/'
model_path = 'model/'

In [33]:
## Load features and labels
## To load TOVS features and labels, replace the prefix 'allfakes' with 'tovs'
data = np.load(os.path.join(intermed_path, 'allfakes_feat.npz'))
label = np.load(os.path.join(intermed_path, 'allfakes_lb.npz'))
data = data['arr_0']
label = label['arr_0']

with open(os.path.join(intermed_path ,'allfakes_lb.pkl'), 'rb') as handle:
       mlb = pickle.load(handle)
num_label = mlb.classes_.shape[0]

print(data.shape)
print(num_label)
print(label.shape)

(3072, 1000)
4
(3072, 4)


## **TRAIN DNN**

In [15]:
def get_in_layer(x, dim_from, dim_to):
   in_layer = tf.keras.layers.Lambda(
       lambda x: x[:, dim_from:dim_to, :])(x)

   return tf.keras.layers.GlobalAveragePooling1D()(in_layer)

In [16]:
def dnn(input_dim, num_labels, vocab_size = 300000, embedding_dim = 300):
   max_len = sum(input_dim)
   inputs = tf.keras.Input(shape=(max_len,), name='input')

   x = tf.keras.layers.Embedding(vocab_size, embedding_dim, 
                                input_length = max_len,
                                name = 'embeddings')(inputs)
   in_layers = []
   in_start = 0
   for in_len in input_dim:
       in_layers.append(get_in_layer(x, in_start, in_start + in_len)) #Calling function for global average pooling
       in_start += in_len    

   #x = tf.keras.layers.concatenate(in_layers) #Concatenating all pooled input embeddings
   x = in_layers.pop()
   
   x = tf.keras.layers.Dropout(0.2)(x)
   x = tf.keras.layers.Dense(1024)(x)
   x = tf.keras.layers.BatchNormalization()(x)
   x = tf.keras.layers.Activation('relu')(x)
   x = tf.keras.layers.Dropout(0.2)(x)
   x = tf.keras.layers.Dense(512)(x)
   x = tf.keras.layers.BatchNormalization()(x)
   x = tf.keras.layers.Activation('relu')(x)

   x = tf.keras.layers.Dropout(0.2)(x)
   output_name = 'output'
   activation_function = 'softmax'
   output_loss = 'categorical_crossentropy'
   output_metric = ['accuracy']

   x_type = tf.keras.layers.Dense(num_labels)(x)
   x_type = tf.keras.layers.BatchNormalization()(x_type)
   x_type = tf.keras.layers.Activation(
               activation_function,
               name=output_name)(x_type)
   output_layers = x_type

   #adam = tf.contrib.opt.LazyAdamOptimizer(learning_rate=0.01, beta1=0.9, beta2=0.999)
   model = tf.keras.Model(inputs=inputs, outputs=output_layers)
   model.compile(loss=output_loss, optimizer='adam', metrics=output_metric)

   return model


In [17]:
epochs = 10
batch_size = 32
vocab_size = 300000
embedding_dim = 100
input_dim = [1000]

In [18]:
model = dnn(input_dim, num_label, vocab_size, embedding_dim)

earlystop_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4, verbose=1, mode='auto')
model.fit(x = data, y = label, batch_size = batch_size, epochs = epochs, shuffle=True)

tf.keras.models.save_model(
       model,
       filepath = os.path.join(model_path, 'dnn_model_weight_allfakes.h5'),
       overwrite=True,
       )


Epoch 1/10
96/96 [==============================] - 17s 169ms/step - loss: 1.2784 - accuracy: 0.4530
Epoch 2/10
96/96 [==============================] - 16s 167ms/step - loss: 0.6537 - accuracy: 0.8003
Epoch 3/10
96/96 [==============================] - 16s 164ms/step - loss: 0.3693 - accuracy: 0.9507
Epoch 4/10
96/96 [==============================] - 15s 160ms/step - loss: 0.2822 - accuracy: 0.9706
Epoch 5/10
96/96 [==============================] - 15s 159ms/step - loss: 0.2392 - accuracy: 0.9756
Epoch 6/10
96/96 [==============================] - 15s 160ms/step - loss: 0.2044 - accuracy: 0.9811
Epoch 7/10
96/96 [==============================] - 15s 159ms/step - loss: 0.1753 - accuracy: 0.9856
Epoch 8/10
96/96 [==============================] - 15s 159ms/step - loss: 0.1526 - accuracy: 0.9888
Epoch 9/10
96/96 [==============================] - 15s 160ms/step - loss: 0.1338 - accuracy: 0.9921
Epoch 10/10
96/96 [==============================] - 15s 159ms/step - loss: 0.1210 - accura

## **TRAIN LSTM**

In [34]:
def get_lstm(input_dim, num_labels, vocab_size = 300000, embedding_dim = 300):

  max_len = sum(input_dim)
  inputs = tf.keras.Input(shape=(max_len,), name='lstm_input')

  x = tf.keras.layers.Embedding(vocab_size, embedding_dim, 
                                input_length = max_len,
                                name = 'embeddings')(inputs)
  
  forward_sent = tf.keras.layers.LSTM(512, dropout = 0.2)(x)
  backward_sent = tf.keras.layers.LSTM(512, dropout=0.2, go_backwards=True)(x)
  sent_encode = tf.keras.layers.Concatenate()([forward_sent, backward_sent])
  sent_encode = tf.keras.layers.Dropout(0.2)(sent_encode)
  output = tf.keras.layers.Dense(num_labels, activation='softmax')(sent_encode)

  model = tf.keras.Model(inputs = inputs, outputs = output)
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  print(model.summary())
  return model

In [43]:
epochs = 10
batch_size = 32
vocab_size = 300000
embedding_dim = 100
input_dim = [1000]

In [44]:
model = get_lstm(input_dim, num_label, vocab_size, embedding_dim)

earlystop_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4, verbose=1, mode='auto')
model.fit(x = data, y = label, batch_size = batch_size, epochs = epochs, shuffle=True)

tf.keras.models.save_model(
       model,
       filepath = os.path.join(model_path, 'lstm_model_weight_allfakes.h5'),
       overwrite=True,
       )

Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
lstm_input (InputLayer)         [(None, 1000)]       0                                            
__________________________________________________________________________________________________
embeddings (Embedding)          (None, 1000, 100)    30000000    lstm_input[0][0]                 
__________________________________________________________________________________________________
lstm_11 (LSTM)                  (None, 512)          1255424     embeddings[0][0]                 
__________________________________________________________________________________________________
lstm_12 (LSTM)                  (None, 512)          1255424     embeddings[0][0]                 
____________________________________________________________________________________________